In [2]:
import numpy as np
import pandas as pd
import openpyxl
import pandas as pd
import scipy as scp
import matplotlib.pyplot as plt

In [3]:
def yearLetter(year):
    A = {
        1995: 'E',
        1996: 'F',
        1997: 'G',
        1998: 'H',
        1999: 'I',
        2000: 'J',
        2001: 'K',
        2002: 'L',
        2003: 'M',
        2004: 'N',
        2005: 'O',
        2006: 'P',
        2007: 'Q',
        2008: 'R',
        2009: 'S'
    }
    return A[year]

In [42]:
def read_excel(filename, sheet, dims):
    wb = openpyxl.load_workbook(filename=filename, 
                   read_only=True)

    ws = wb[sheet]

    data_rows = []
    for row in ws[dims[0]:dims[1]]:
        data_cols = []
        for cell in row:
            data_cols.append(cell.value)
        data_rows.append(data_cols)
    return np.array(data_rows)

In [327]:
def StaticModelConstruction(year):
    IOTableWorksheet = str(year);
    L = yearLetter(year);
    
    IO = read_excel('JPN_NIOT.xlsx', IOTableWorksheet, ('E7','AL40'))
    ImportsIO = read_excel('JPN_NIOT.xlsx', IOTableWorksheet, ('E42','AL76'))
    ValueAdded = read_excel('JPN_NIOT.xlsx', IOTableWorksheet, ('E78','AL83'))
    Taxes = read_excel('JPN_NIOT.xlsx', IOTableWorksheet, ('AN78','AR83'))
    Consumption = read_excel('JPN_NIOT.xlsx', IOTableWorksheet, ('AN7','AR40'))
    Exports = read_excel('JPN_NIOT.xlsx', IOTableWorksheet, ('AS7','AS40'))
    Output = read_excel('JPN_NIOT.xlsx', IOTableWorksheet, ('AT7','AT40'))
    Compensation = read_excel('Compensation.xlsx', 'DATA', (L+'19911',L+'19944'))
    Rate = read_excel('Exchange.xlsx', 'EXR', ('C27','Q27'))
    JPNGDP = read_excel('JPN_GDP.xlsx','Data',('E2','S2'))

    P = IO;
    V = np.sum(ValueAdded, axis=0)
    Y = np.sum(Consumption, axis=1)
    Tx0 = np.sum(Taxes, axis=0)
    Tx = np.sum(Tx0);
    Im = np.sum(ImportsIO, axis=0);
    Ex = Exports;
    I = Output;
    realGDP = JPNGDP*1000*Rate;
    W = Compensation*Rate;
    GDP = np.sum(V) + Tx;
    GDPError = abs(realGDP-GDP)/realGDP;
    n = np.shape(I)[0];
    Iw = I - np.asmatrix(Im).H;
    Yw = Y + Ex - np.asmatrix(Im).H;
    Yw = np.diagonal(Yw).reshape(-1,1)
    R = np.zeros((n,n));
    for i in range(n):
        R[:,i] = P[:,i]/Iw[i,0];
    Iw = list(Iw)
    Iw.append([GDP])
    Iw = np.asmatrix(Iw)
    Vwr = np.divide(V, np.asmatrix(Iw[0:n]).H);
    Wwr = np.divide(V, np.asmatrix(Iw[0:n]).H);
    Ywr = Yw/GDP;
    Txr = Tx/GDP;
    Rw1 = list(Ywr)
    Rw1.append([Txr])
    Rw1 = np.array(Rw1)
    Rw0 = np.hstack((R, Ywr))
    Rw1 = Rw1.T
    Rw = np.vstack((Rw0,Rw1))
    StaticModelError = np.linalg.norm(Iw-Rw*Iw);
                  
    return (Iw, Rw, P, V, Y, W, Rate, GDPError, StaticModelError) 

In [352]:
def aggregation(Iw, Rw, aggregateIndustriesNumber):
    print(aggregateIndustriesNumber)
    m = np.max(aggregateIndustriesNumber)
    aggregateIndustry = [0 for i in range(m)]
    print(m)
    Ia = np.zeros((m,1))
    Ra = np.zeros((m,m))
    industryAmount = 0
    for i in range(m):
        NB = np.where(aggregateIndustriesNumber == i)[0]
        if (len(NB) != 0):
            aggregateIndustry[i] = NB[0]
        Ia[i,0] = np.sum(Iw[aggregateIndustry[i],0])
        #industryAmount = industryAmount + len(aggregateIndustry[i])
    print(aggregateIndustry)
    for i in range(m):
        for j in range(m):
            R1 = Rw[aggregateIndustry[i],aggregateIndustry[j]]
            I1 = Iw[aggregateIndustry[j]]
            print(R1)
            Ra[i,j] = np.sum(R1*I1)/Ia[j,0]
    return (Ra, Ia)

In [356]:
def reduction(Iw, Rw, unreducedIndustriesNumbers):
    n = np.shape(Iw)[0]-1
    reducedIndustriesNumbers = np.setdiff1d([i for i in range(1,n+2)], unreducedIndustriesNumbers)
    n1 = len(reducedIndustriesNumbers)
    R11 =
    for i in range(n1):
        
    R11 = Rw[unreducedIndustriesNumbers,unreducedIndustriesNumbers]
    R12 = Rw[unreducedIndustriesNumbers,reducedIndustriesNumbers]
    R21 = Rw[reducedIndustriesNumbers,unreducedIndustriesNumbers]
    R22 = Rw[reducedIndustriesNumbers,reducedIndustriesNumbers]
    
    Ir = Iw[unreducedIndustriesNumbers,0]
    Rr = R11 + R12 * np.linalg.inv(np.eye(n1)-R22) * R21
    return (Ir, Rr)

In [13]:
def IndexesComputation(Iw, V, Y, W, Vb, Wb, rf):
    rg = Vb/Iw[-1];
    Cn = Vb + np.sum(W)*(1-rf);
    Rc = Cn/np.sum(V);
    rw = np.sum(W)/np.sum(V);
    Rh = 1 - Rc;
    Prb = Vb - Wb;
    Pcb = np.sum(Y)+Wb;
    Rntb = Prb/Pcb;
    Rsb = Pcb/Iw[-1];
    
    return (rg, Cn, Rc, rw, Rh, Prb, Pcb, Rhtb, Rsb)

In [14]:
def EfficiencyAnalysis(Ra, Ia, W):
    Rp = Ra[1,1]
    Vr = Ra[1,2]
    Is = Ia[1,1]
    Wr = np.sum(W)/Is
    Prr = Vr - Wr
    Rsp = Rp + Wr
    Rnt = Prr/Rsp
    return (Prr, Rsp, Rnt)
    

In [15]:
def DifferentialModelConstruction(I, ILast, R):
    M = np.diag((I-ILast)/I)
    D = M*R

In [16]:
def DifferenceModelConstruction(I, ILast, R):
    M = np.eye(np.shape(I)[0])+np.diag((I-ILast)/I)
    D = M*R

In [17]:
def DifferentialModelPrognosis(I,D):
    n,m = np.shape(I);
    pI = np.zeros(n,m-1);
    pError = np.zeros(n,m-1);
    for i in range(m-1):
        pI[:,i] = scp.linalg.expm(D[:,:,i]*I[:,i])
        pError[:,i] = abs(pI[:,i]-I[:,i+1])/I[:,i+1]
    pErrorMean = np.mean(pError, axis=1);
    return (pI,pError,pErrorMean)

In [18]:
def DifferenceModelPrognosis(I,D):
    n,m = np.shape(I);
    pI = np.zeros(n,m-1);
    pError = np.zeros(n,m-1);
    for i in range(m-1):
        pI[:,i] = D[:,:,i]*I[:,i]
        pError[:,i] = abs(pI[:,i]-I[:,i+1])/I[:,i+1]
    pErrorMean = np.mean(pError, axis=1);
    return (pI,pError,pErrorMean)

In [357]:
Iw2007,Rw2007,P2007,V2007,Y2007,W2007,RubUSDRate2007,GDPError2007, StaticModelError2007 = StaticModelConstruction(2007)
n = np.shape(Iw2007)[0]-1
l = [1 for i in range(n)]
l.append(2)
print(Iw2007, Rw2007)
Ia2007,Ra2007 = aggregation(Iw2007,Rw2007,l)
Ir2007,Rr2007 = reduction(Iw2007,Rw2007,[20,35])
roubleVb = [1321903, 2012151, 2358546, 2698867, 3512183, 4284803, 5983036]
roubleWb = [258500, 422600, 490500, 637900, 817000, 1057900, 1327600]
Vb2007 = roubleVb(end)*RubUSDRate2007
Wb2007 = roubleWb(end)*RubUSDRate2007
tax = 0.1; # Минимальная ставка подоходного налога в японии
rg,Cn,Rc,rw,Rh,Prb,Pcb,Rntb,Rsb = IndexesComputation(Iw2007,V2007,Y2007,W2007,Vb2007,Wb2007,tax)
Prr,Rsp,Rnt = EfficiencyAnalysis(Ra2007,Ia2007,W2007)

Iw = np.zeros(n+1,7);
Rw = np.zeros(n+1,n+1,7);
Dw = np.zeros(n+1,n+1,6);
Ia = np.zeros(2,7);
Ra = np.zeros(2,2,7);
Da = np.zeros(2,2,6);
Ir = np.zeros(7,7);
Rr = np.zeros(7,7,7);
Dr = np.zeros(7,7,6);
Dl = np.zeros(7,7,6);

for i in range(7):
    Iw[:,i], Rw[:,:,i], *_ = StaticModelConstruction(2000+i);
    Ia[:,i], Ra[:,:,i] = aggregation(Iw[:,i],Rw[:,:,i], [[np.ones(n)], [2]]);
    Ir[:,i], Rr[:,:,i] = reduction(Iw[:,i],Rw[:,:,i], np.array([[2], [8], [18], [20], [30], [31], [35]]));
    if i != 0:
        Dw[:,:,i-1] = DifferentialModelConstruction(Iw[:,i], Iw[:,i-1], Rw[:,:,i]);
        Da[:,:,i-1] = DifferentialModelConstruction(Ia[:,i], Ia[:,i-1], Ra[:,:,i]);
        Dr[:,:,i-1] = DifferentialModelConstruction(Ir[:,i], Ir[:,i-1], Rr[:,:,i]);
        Dl[:,:,i-1] = DifferenceModelConstruction(Ir[:,i], Ir[:,i-1], Rr[:,:,i]);
        

pIw,pIwError,pIwErrorMean = DifferentialModelPrognosis(np.array([Iw[:,2:-1]]).T, Dw);
pIa,pIaError,pIaErrorMean = DifferentialModelPrognosis(np.array(Ia[:,2:-1]).T, Da);
pIr,pIrError,pIrErrorMean = DifferentialModelPrognosis(np.array(Ir[:,2:-1]).T, Dr);
pIl,pIlError,pIlErrorMean = DifferenceModelPrognosis(np.array(Ir[:,2:-1]).T, Dl);

[[104094.73169158115]
 [19824.889001842726]
 [298333.15500133997]
 [36745.427316534835]
 [4032.886658753389]
 [38954.173210468405]
 [117614.72086704329]
 [89936.8319333813]
 [248332.520636277]
 [120415.81415695106]
 [62171.41162385906]
 [478126.43933209765]
 [234781.8603408884]
 [368022.51411834045]
 [479544.1896618755]
 [33728.42333452255]
 [180656.4046627278]
 [565952.3113916534]
 [105716.9931713829]
 [532112.6408248]
 [257732.48948278077]
 [280599.29574460984]
 [187292.56912589178]
 [56256.07726233722]
 [29009.635799443724]
 [57097.88689201804]
 [148981.70924599786]
 [404478.99822300486]
 [575060.3280926386]
 [567898.6987477328]
 [484709.18495781947]
 [185281.15233521326]
 [358684.95546971745]
 [336574.9395758153]
 [4370266.608260001]] [[ 8.93562609e-02  6.24274689e-04  1.61753233e-01 ...  2.12615969e-03
   1.47864116e-03  6.99578611e-03]
 [ 1.26432640e-04  6.25557866e-02  3.43939566e-05 ...  2.45294950e-04
   4.20795481e-05 -1.42809704e-03]
 [ 9.25786539e-02  1.98555674e-03  1.5580

TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [53]:
A = np.array([[2, 3], [6, 8]])
C = np.asmatrix(A).H
print(C)

[[2 6]
 [3 8]]


In [25]:
a, b, *c = [2,3,5,6,7,8,9]
print(a,b)

2 3


In [251]:
A = np.array([[1,2],[3,4]])
B = np.array([[5],[6]])
print(B)
C = np.hstack((A,B))
print(C)

[[5]
 [6]]
[[1 2 5]
 [3 4 6]]


In [290]:
A = np.array([[1],[2],[3]])
print(A)
B = A.T
print(B)

[[1]
 [2]
 [3]]
[[1 2 3]]


In [349]:
arr = np.array([4,8,7])
c = len(np.where(arr==5)[0])
print(c)

0


In [359]:
A = [[1,2,3],[4,5,6],[7,8,9]]
B = []
L = [1,3]
R = [1,3]
for i in L:
    for j in R: 
           B[i,j] = A[i,j]
print(B)

TypeError: cannot unpack non-iterable int object